In [60]:
#importing necessary libraries
import pandas as pd
import sqlite3

#Establishing connection to database
conn = sqlite3.connect("pharm_data.db")

#Fetching list of tables we have access to, for reference
cursor = conn.cursor()
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("ALL TABLES: ",end="")
for table in tables: print(table[0],end=", ")
print("\b\b")

#Asks user for their zip code, gets state, determines pdp_region_code based on state
user_zip = input("Enter your zipcode: ")
user_state = pd.read_sql("select COUNTY,STATENAME\
                          from ZIP_STATE_COUNTY\
                          where ZIPCODE is '" + user_zip + "'", conn)\
            ['STATENAME'].values[0].rstrip()

user_pdp_region_code = pd.read_sql("select distinct PDP_REGION_CODE\
                                    from GEOGRAPHIC_LOCATOR\
                                    where STATENAME is '"+user_state+"'",conn)\
                       ['PDP_REGION_CODE'].values[0]

ALL TABLES: BENEFICIARY_COST, GEOGRAPHIC_LOCATOR, PLAN_INFO, ZIP_STATE_COUNTY, NCPDP, FORMULARY_DATA, NDC_DATA
Enter your zipcode: 07095


In [78]:
#Step 3, filtering formulary_ids by pdp_region_code and contract_id
F_SUBSET = pd.read_sql("select * from PLAN_INFO\
                        where PDP_REGION_CODE is '"+str(user_pdp_region_code)+"'\
                        and CONTRACT_ID like 'S%'",conn)\
           [['CONTRACT_ID','PLAN_ID','SEGMENT_ID','FORMULARY_ID','PREMIUM','DEDUCTIBLE','PDP_REGION_CODE']]
F_SUBSET

,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,PDP_REGION_CODE
0,S0522,4,0,18041,30.4,405.0,4
1,S4802,78,0,18443,36.9,405.0,4
2,S4802,101,0,18439,79.5,0.0,4
3,S5601,8,0,18419,34.3,0.0,4
4,S5601,9,0,18420,84.6,0.0,4
5,S5617,18,0,18082,74.8,405.0,4
6,S5617,249,0,18083,52.9,0.0,4
7,S5660,106,0,18154,34.0,405.0,4
8,S5660,207,0,18155,91.7,350.0,4
9,S5660,220,0,18152,22.6,405.0,4


In [86]:
#Step 4, asking user for drugs and strengths
user_generics = {(input("Enter a generic drug name: ").upper(),\
                  float(input("Enter a desired strength: "))):[]\
                 for i in range(int(input("Enter how many drugs you wish to search: ")))}
user_generics

Enter how many drugs you wish to search: 2
Enter a generic drug name: Lisinopril
Enter a desired strength: 10
Enter a generic drug name: Somatropin
Enter a desired strength: 0


{('LISINOPRIL', 10.0): [], ('SOMATROPIN', 0.0): []}

In [188]:
#getting NDCs for each drug,strength combo the user offered in step 4
for drug,strength in user_generics:
    user_generics[(drug,strength)] = pd.read_sql("select PRODUCTNDC from NDC_DATA\
                 where upper(NONPROPRIETARYNAME) like '%"+drug+"%'\
                 and (ACTIVE_NUMERATOR_STRENGTH is '' or ACTIVE_NUMERATOR_STRENGTH is "+str(strength)+")",conn)\
    .applymap(lambda s: "0"*(11-(len(s)-1))+s.replace("-",""))

In [189]:
for drug,strength in user_generics:
    print("DRUG: %s, STRENGTH: %s\n%s" % (drug,strength,user_generics[drug,strength]))

DRUG: LISINOPRIL, STRENGTH: 10.0
      PRODUCTNDC
0    00000060106
1    00001431267
2    00001439714
3    00001723759
4    00001850101
5    00001850610
6    00003100131
7    00003782074
8    00005910407
9    00006034211
10   00006155521
11   00006157641
12   00006157776
13   00006157917
14   00006158053
15   00006158063
16   00009045808
17   00009046485
18   00010544238
19   00010544905
20   00016590306
21   00016729377
22   00021695329
23   00031722419
24   00033261332
25   00033358212
26   00043063485
27   00043063769
28   00043353113
29   00043353169
..           ...
86   00672960622
87   00672961249
88   00068001210
89   00068001268
90   00068001334
91   00680711672
92   00680713113
93   00680713122
94   00068084197
95   00068180514
96   00068180980
97   00068405070
98   00068645552
99   00687886407
100  00687886782
101  00687888912
102  00687888944
103  00687889411
104  00687889661
105  00687889823
106  00687889912
107  00705180246
108  00705180402
109  00705180468
110  0070518053

In [164]:
drug = 'LISINOPRIL'
strength = 10
pd.read_sql("select * from FORMULARY_DATA\
 where NDC in (select PRODUCTNDC from NDC_DATA\
                 where upper(NONPROPRIETARYNAME) like '%"+drug+"%'\
                 and (ACTIVE_NUMERATOR_STRENGTH is '' or ACTIVE_NUMERATOR_STRENGTH is "+str(strength)+"))",conn)
    

,FORMULARY_ID,FORMULARY_VERSION,CONTRACT_YEAR,RXCUI,NDC,TIER_LEVEL_VALUE,QUANTITY_LIMIT_YN,QUANTITY_LIMIT_AMOUNT,QUANTITY_LIMIT_DAYS,PRIOR_AUTHORIZATION_YN,STEP_THERAPY_YN


In [186]:
pd.read_sql("select * from FORMULARY_DATA",conn)

,FORMULARY_ID,FORMULARY_VERSION,CONTRACT_YEAR,RXCUI,NDC,TIER_LEVEL_VALUE,QUANTITY_LIMIT_YN,QUANTITY_LIMIT_AMOUNT,QUANTITY_LIMIT_DAYS,PRIOR_AUTHORIZATION_YN,STEP_THERAPY_YN
0,FORMULARY_ID,FORMULARY_VERSION,CONTRACT_YEAR,RXCUI,NDC,TIER_LEVEL_VALUE,QUANTITY_LIMIT_YN,QUANTITY_LIMIT_AMOUNT,QUANTITY_LIMIT_DAYS,PRIOR_AUTHORIZATION_YN,STEP_THERAPY_YN
1,00018000,9,2018,1551300,00002143380,2,Y,2,28,N,N
2,00018000,9,2018,1551306,00002143480,2,Y,2,28,N,N
3,00018000,9,2018,1745108,00002144511,2,N,,,Y,N
4,00018000,9,2018,855820,00002512130,2,Y,30,30,N,N
5,00018000,9,2018,855816,00002512330,2,Y,30,30,N,N
6,00018000,9,2018,207834,00002733511,2,N,,,Y,N
7,00018000,9,2018,1728075,00002762301,2,N,,,N,N
8,00018000,9,2018,1718936,00002763511,2,Y,2,28,N,N
9,00018000,9,2018,1657777,00002766901,2,N,,,Y,N


In [185]:
CLEAN_NDC_DATA_TABLE = pd.read_sql("select PRODUCTNDC from NDC_DATA",conn).applymap(lambda s: s.replace("-","")).iloc[1:]
def clean_ndc(s):
    if len(s) in [11,10,9]:
        
CLEAN_NDC_FD_TABLE = pd.read_sql("select NDC from FORMULARY_DATA",conn).applymap()
pd.read_sql("select PRODUCTNDC from CLEAN_NDC_DATA_TABLE where PRODUCTNDC is '00060106'",conn)

,PRODUCTNDC
0,0006-0106
